<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-8e680c40b9a0300e'
PROJECT = 'qwiklabs-gcp-8e680c40b9a0300e'
REGION = 'us-east1-c'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,1403073183891835564,351299
1,7170969733900686954,331274
2,7146494315947640619,335327
3,8029892925374153452,339209
4,1451354159195218418,334485


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [18]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13296 examples in the train dataset and 3260 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [19]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.300404,True,34,1,38.0,774501970389208065
1,6.874013,False,24,1,35.0,774501970389208065
2,7.251004,True,30,1,38.0,774501970389208065
3,7.687519,True,23,1,39.0,774501970389208065
4,8.501025,True,39,1,39.0,774501970389208065


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [20]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13282.000000,13296.000000,13296.000000,13203.000000,1.329600e+04
mean,7.223674,27.460590,1.036628,38.613573,4.379693e+18
std,1.314060,6.214998,0.195312,2.551937,2.793910e+18
min,0.500449,12.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,15.000252,51.000000,3.000000,47.000000,9.183606e+18


In [21]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [22]:
print(traindf.head())# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
print(traindf.head())

   weight_pounds  is_male  mother_age  plurality  gestation_weeks  \
0       8.300404     True          34          1             38.0   
1       6.874013    False          24          1             35.0   
2       7.251004     True          30          1             38.0   
3       7.687519     True          23          1             39.0   
4       8.501025     True          39          1             39.0   

            hashmonth  
0  774501970389208065  
1  774501970389208065  
2  774501970389208065  
3  774501970389208065  
4  774501970389208065  
   weight_pounds is_male  mother_age  plurality  gestation_weeks  \
0       8.300404    True          34  Single(1)             38.0   
1       6.874013   False          24  Single(1)             35.0   
2       7.251004    True          30  Single(1)             38.0   
3       7.687519    True          23  Single(1)             39.0   
4       8.501025    True          39  Single(1)             39.0   

            hashmonth  
0  77450

In [23]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13291,1.812200,Unknown,24,Single(1),27.0,6637442812569910270
13292,7.936641,Unknown,20,Single(1),39.0,6637442812569910270
13293,6.499227,Unknown,26,Single(1),37.0,6637442812569910270
13294,6.812284,Unknown,20,Single(1),35.0,6637442812569910270
13295,7.187070,Unknown,23,Single(1),40.0,6637442812569910270


In [24]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26386.000000,26386.000000,26386.000000,2.638600e+04
mean,7.223258,27.457667,38.622224,4.377186e+18
std,1.312842,6.213086,2.522608,2.792624e+18
min,0.500449,12.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,15.000252,51.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [25]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [26]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6460 eval.csv
  26386 train.csv
  32846 total
==> eval.csv <==
8.50102482272,True,33,Single(1),38.0,1639186255933990135
9.31232594688,True,33,Single(1),38.0,6782146986770280327
1.4550509292,True,28,Single(1),25.0,6392072535155213407
8.50763869058,True,36,Single(1),39.0,3182182455926341111
7.3744626639,False,22,Single(1),39.0,1639186255933990135
7.7823178486,False,25,Single(1),39.0,6244544205302024223
7.76468086764,False,28,Single(1),34.0,1569531340167098963
7.04597389352,True,20,Single(1),32.0,7517141034410775575
8.75014717878,True,32,Single(1),41.0,270792696282171059
6.3493131456,True,35,Single(1),35.0,7517141034410775575

==> train.csv <==
8.3004041643,True,34,Single(1),38.0,774501970389208065
6.87401332916,False,24,Single(1),35.0,774501970389208065
7.25100379718,True,30,Single(1),38.0,774501970389208065
7.68751907594,True,23,Single(1),39.0,774501970389208065
8.50102482272,True,39,Single(1),39.0,774501970389208065
8.375361333379999,False,35,Single(1),40.0,774501970389208065
7.4361

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License